In [ ]:
import os
import librosa
import numpy as np

def load_wav_files_from_directory(directory_path):
    audio_files = []
    sr = None  # Initialize sr with a default value
    for filename in os.listdir(directory_path):
        if filename.endswith(".wav"):
            filepath = os.path.join(directory_path, filename)
            audio, sr = librosa.load(filepath, sr=None)
            audio_files.append(audio)
    if sr is None:  # Handle the case where no .wav files are found
        raise ValueError(f"No .wav files found in directory {directory_path}")
    return audio_files, sr

noisy_train_dir = r"C:\Users\Asus\OneDrive\Documents\python\Noise cancellation\data\noisy_train"
clean_train_dir = r"C:\Users\Asus\OneDrive\Documents\python\Noise cancellation\data\clean_train"
noisy_test_dir = r"C:\Users\Asus\OneDrive\Documents\python\Noise cancellation\data\noisy_test"
clean_test_dir = r'C:\Users\Asus\OneDrive\Documents\python\Noise cancellation\data\clean_test'

noisy_train_files, sr = load_wav_files_from_directory(noisy_train_dir)
clean_train_files, sr = load_wav_files_from_directory(clean_train_dir)
noisy_test_files, sr = load_wav_files_from_directory(noisy_test_dir)
clean_test_files, sr = load_wav_files_from_directory(clean_test_dir)


In [ ]:
def get_max_stft_length(audio_files, sr):
    max_length = 0
    for audio in audio_files:
        stft_audio = librosa.stft(audio)
        if stft_audio.shape[1] > max_length:
            max_length = stft_audio.shape[1]
    return max_length

max_length = max(get_max_stft_length(noisy_train_files, sr),
                 get_max_stft_length(clean_train_files, sr),
                 get_max_stft_length(noisy_test_files, sr),
                 get_max_stft_length(clean_test_files, sr))


In [ ]:
def preprocess_audio_files(audio_files, sr, max_length):
    magnitudes = []
    phases = []
    for audio in audio_files:
        audio = librosa.util.normalize(audio)
        stft_audio = librosa.stft(audio)
        magnitude, phase = librosa.magphase(stft_audio)
        
        # Pad the magnitude and phase to the max_length
        magnitude_padded = np.pad(magnitude, ((0, 0), (0, max_length - magnitude.shape[1])), mode='constant')
        phase_padded = np.pad(phase, ((0, 0), (0, max_length - phase.shape[1])), mode='constant')
        
        magnitudes.append(magnitude_padded)
        phases.append(phase_padded)
        
    return np.array(magnitudes), np.array(phases)

noisy_train_mag, noisy_train_phase = preprocess_audio_files(noisy_train_files, sr, max_length)
clean_train_mag, clean_train_phase = preprocess_audio_files(clean_train_files, sr, max_length)
noisy_test_mag, noisy_test_phase = preprocess_audio_files(noisy_test_files, sr, max_length)
clean_test_mag, clean_test_phase = preprocess_audio_files(clean_test_files, sr, max_length)


In [ ]:
print("noisy_train_mag shape:", noisy_train_mag.shape)
print("clean_train_mag shape:", clean_train_mag.shape)
print("noisy_test_mag shape:", noisy_test_mag.shape)
print("clean_test_mag shape:", clean_test_mag.shape)

In [ ]:
X_train = noisy_train_mag[:, :, np.newaxis]
y_train = clean_train_mag[:, :, np.newaxis]
X_test = noisy_test_mag[:, :, np.newaxis]
y_test = clean_test_mag[:, :, np.newaxis]


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation

def build_cnn_model(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv2D(64, (3, 3), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(64, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    outputs = Conv2D(1, (3, 3), padding='same')(x)
    model = Model(inputs, outputs)
    return model

input_shape = (noisy_train_mag.shape[1], noisy_train_mag.shape[2], 1)
model = build_cnn_model(input_shape)
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))


In [ ]:
def save_cleaned_audio(magnitude, phase, sr, output_filepath):
    stft_audio = magnitude * phase
    cleaned_audio = librosa.istft(stft_audio)
    librosa.output.write_wav(output_filepath, cleaned_audio, sr)

# Predict and save the cleaned audio for the test set
clean_test_pred = model.predict(X_test)
clean_test_pred = clean_test_pred[..., 0]

for i in range(len(clean_test_pred)):
    output_filepath = f"cleaned_test_audio_{i}.wav"
    save_cleaned_audio(clean_test_pred[i], noisy_test_phase[i], sr, output_filepath)


In [ ]:
import os
import librosa
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation

def load_wav_files_from_directory(directory_path):
    audio_files = []
    for filename in os.listdir(directory_path):
        if filename.endswith(".wav"):
            filepath = os.path.join(directory_path, filename)
            audio, sr = librosa.load(filepath, sr=None)
            audio_files.append(audio)
    return audio_files, sr

def preprocess_audio_files(audio_files, sr, max_length):
    magnitudes = []
    phases = []
    for audio in audio_files:
        audio = librosa.util.normalize(audio)
        stft_audio = librosa.stft(audio)
        magnitude, phase = librosa.magphase(stft_audio)
        
        # Pad the magnitude and phase to the max_length
        magnitude_padded = np.pad(magnitude, ((0, 0), (0, max_length - magnitude.shape[1])), mode='constant')
        phase_padded = np.pad(phase, ((0, 0), (0, max_length - phase.shape[1])), mode='constant')
        
        magnitudes.append(magnitude_padded)
        phases.append(phase_padded)
        
    return np.array(magnitudes), np.array(phases)

def build_cnn_model(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv2D(64, (3, 3), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(64, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    outputs = Conv2D(1, (3, 3), padding='same')(x)
    model = Model(inputs, outputs)
    return model

noisy_train_dir = r"C:\Users\Asus\OneDrive\Documents\python\Noise cancellation\Dataset\noisy_trainset_wav"
clean_train_dir = r"C:\Users\Asus\OneDrive\Documents\python\Noise cancellation\Dataset\clean_trainset_wav"
noisy_test_dir = r"C:\Users\Asus\OneDrive\Documents\python\Noise cancellation\Dataset\noisy_testset_wav"
clean_test_dir = r"C:\Users\Asus\OneDrive\Documents\python\Noise cancellation\Dataset\clean_testset_wav"

noisy_train_files, sr = load_wav_files_from_directory(noisy_train_dir)
clean_train_files, sr = load_wav_files_from_directory(clean_train_dir)
noisy_test_files, sr = load_wav_files_from_directory(noisy_test_dir)
clean_test_files, sr = load_wav_files_from_directory(clean_test_dir)

max_length = 674  # Set this to the maximum length of the STFT

noisy_train_mag, noisy_train_phase = preprocess_audio_files(noisy_train_files, sr, max_length)
clean_train_mag, clean_train_phase = preprocess_audio_files(clean_train_files, sr, max_length)
noisy_test_mag, noisy_test_phase = preprocess_audio_files(noisy_test_files, sr, max_length)
clean_test_mag, clean_test_phase = preprocess_audio_files(clean_test_files, sr, max_length)

# Reshape the data correctly
X_train = noisy_train_mag[..., np.newaxis]
y_train = clean_train_mag[..., np.newaxis]
X_test = noisy_test_mag[..., np.newaxis]
y_test = clean_test_mag[..., np.newaxis]

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

# Check the input shape
input_shape = (X_train.shape[1], X_train.shape[2], 1)
model = build_cnn_model(input_shape)
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))
